In [1]:
import pandas as pd
import numpy as np
from datetime import date
import datetime as dt

In [25]:
#将数据分为3个数据集 利用滑窗法
#将2016年1月1日到4月13日的数据提取特征，利用4月14日的到5月14日的作为测试集
#将2月1日到5月14日的作为数据集提取特征，利用5月15日6月15日的作为测试集
#将3月15日到6月30日作为数据集提取特征，再测试7月1日到7月31日的数据


#1 提取用户特征
#    距离
#    用户的平均距离，用户的最小距离，用户的最大距离
#    使用优惠券买的物品数，买的总数，收到的优惠券数
#    使用优惠券买的/总共收到的优惠券

In [5]:
#利用pandas读取csv个格式的数据,header=None表示原文件没有索引
#原文件中总共有1754884个记录,header=0，表明第0行代表列名

In [13]:
off_train = pd.read_csv('data/ccf_offline_stage1_train.csv')
off_train.columns=['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_train.head(2)

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN


In [36]:
off_test = pd.read_csv("data/ccf_offline_stage1_test_revised.csv",header=0)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
off_test.head(2)

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706


In [37]:
on_train = pd.read_csv("data/ccf_online_stage1_train.csv",header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
on_train.head(2)

,user_id,merchant_id,action,coupon_id,discount_rate,date_received,date
0,13740231,18907,2,100017492,500:50,20160513.0,NaN
1,13740231,34805,1,NaN,NaN,NaN,20160321.0


In [30]:
receive_coupon_use_count = off_train[off_train.date_received.notnull() & off_train.date.notnull()].shape[0]
receive_coupon_not_use_count = off_train[off_train.date_received.notnull() & off_train.date.isnull()].shape[0]
not_receive_coupon_buy_count = off_train[off_train.date_received.isnull() & off_train.date.notnull()].shape[0]
not_receive_coupon_not_buy = off_train[off_train.date_received.isnull() & off_train.date.isnull()].shape[0]

In [35]:
print('获得优惠卷并使用的人数:', receive_coupon_use_count)
print('获得优惠卷没使用的人数:', receive_coupon_not_use_count)
print('没获得优惠卷并购物的人数:', not_receive_coupon_buy_count)
print('没获得优惠卷没购物的人数:', not_receive_coupon_not_buy)

获得优惠卷并使用的人数: 75382
获得优惠卷没使用的人数: 977900
没获得优惠卷并购物的人数: 701602
没获得优惠卷没购物的人数: 0


In [42]:
#使数据集3等于test集
dataset3 = off_test
#数据集3的特征为 取 线上数据中领券和用券日期大于3月15日和小于6月30日的
feature3 = off_train[((off_train.date>=20160315)&(off_train.date<=20160630))|((off_train.date.isnull())&(off_train.date_received>=20160315)&(off_train.date_received<=20160630))]
#提取数据集2的测试集
dataset2 = off_train[(off_train.date_received>=20160515)&(off_train.date_received<=20160615)]
#在2月1日到5月14日之间使用了券,只要领取时间在2月1日到5月14日之间,并包括没有数据中没有领取券的
feature2 = off_train[(off_train.date>=20160201)&(off_train.date<=20160514)|((off_train.date.isnull())&(off_train.date_received>=20160201)&(off_train.date_received<=20160514))]

#同理可得
dataset1 = off_train[(off_train.date_received>=201604014)&(off_train.date_received<=20160514)]
feature1 = off_train[(off_train.date>=20160101)&(off_train.date<=20160413)|((off_train.date.isnull())&(off_train.date_received>=20160101)&(off_train.date_received<=20160413))]

In [38]:
#这里用到的特征有
#    用户领取的所有优惠券数目
#    ◦用户领取的特定优惠券数目
#    ◦用户此次之后/前领取的所有优惠券数目
#    ◦用户此次之后/前领取的特定优惠券数目
#    ◦用户上/下一次领取的时间间隔
#    ◦用户领取特定商家的优惠券数目
#    ◦用户领取的不同商家数目
#    ◦用户当天领取的优惠券数目
#    ◦用户当天领取的特定优惠券数目
#    ◦用户领取的所有优惠券种类数目
#    ◦商家被领取的优惠券数目
#    ◦商家被领取的特定优惠券数目
#    ◦商家被多少不同用户领取的数目
#    ◦商家发行的所有优惠券种类数目
#这些特征都是在预测集中被提取的

In [55]:
#t代表数据的特征t ,t1 ,t2........


#对于测试集3，提取用户的Id
t = dataset3[['user_id']].copy()
#相当于给原有数据加上一列，这个月用户收取的所有优惠券数目，并初始化为1
t['this_month_user_receive_all_coupon_count'] = 1

#将t按照用户id进行分组，然后统计所有用户收取的优惠券数目,并初始化一个索引值
t = t.groupby('user_id').agg('sum').reset_index()

In [58]:
#提取数据集3的优惠券Id和用户Id
t1 = dataset3[['user_id','coupon_id']].copy()
#提取这个月用户收到的相同的优惠券的数量
t1['this_month_user_receive_same_coupn_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()